Imports

In [ ]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_DID = "did:nv:519f378cabc1860e8baa8c90c529405eae4fa7884916a274515ef66ef9e49a02"
print(ASSET_DID)

Setup Nevermined and accounts

In [ ]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


Publish algorithm

In [ ]:
metadata_algorithm = {
    "main": {
        "name": "Fraud Demo Algorithm",
        "dateCreated": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "author": "Nevermined Consumer",
        "license": "",
        "price": "0",
        "files": [
            {
                "index": 0,
                "contentType": "text/text",
                "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
                "checksumType": "MD5",
                "contentLength": "12057507",
                "url": "https://raw.githubusercontent.com/nevermined-io/fl-demo/feature/xain-fraud-detection/fraud-detection-sherpa/notebooks/federated_fraud_demo.ipynb"
            }
        ],
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "py",
            "version": "0.1.0",
            "entrypoint": (
                "pip install jupyter shfl pandas papermill && "
                "papermill --stdout-file - federated_fraud_demo.ipynb $NEVERMINED_OUTPUTS_PATH/federated_fraud_demo_output.ipynb"
            ),
            "requirements": {
                "container": {
                    "image": "python",
                    "tag": "3.8-slim-buster",
                    "checksum": "sha256:53ad3a03b2fb240b6c494339821e6638cd44c989bcf26ec4d51a6a52f7518c1d"
                }
            }           
        }
    }
}

ddo_algorithm = nevermined.assets.create(metadata_algorithm, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm.did}")

Publish the workflow

In [ ]:
metadata_workflow = {
    "main": {
        "name": "Fraud Demo Workflow",
        "dateCreated": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "author": "Nevermined Consumer",
        "license": "",
        "price": "0",
        "type": "workflow",
        "workflow": {
            "stages": [
                {
                    "index": 0,
                    "input": [
                        {
                            "index": 0,
                            "id": ASSET_DID
                        }
                    ],
                    "transformation": {
                        "id": ddo_algorithm.did
                    }
                }
            ]
        }
    }
}

ddo_workflow = nevermined.assets.create(metadata_workflow, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow.did}")

Order computation

In [ ]:
service_agreement_id = nevermined.assets.order(ASSET_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id}")


Wait for payment to be processed by the blockchain

In [ ]:
keeper = Keeper.get_instance()
event = keeper.lock_reward_condition.subscribe_condition_fulfilled(
        service_agreement_id, 60, None, (), wait=True
    )
assert event is not None, "Reward condition is not found"

Execute Workflow

In [ ]:
execution_id = nevermined.assets.execute(
    service_agreement_id,
    ASSET_DID,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow.did
)

print(f"Execution ID: {execution_id}")

Wait for compute job to finish

In [ ]:
status = None
while status not in ["Succeeded", "Failed"]:
    result = nevermined.assets.compute_status(service_agreement_id, execution_id, consumer_account)
    status = result["status"]
    print(f"{execution_id}: {status}")
    time.sleep(10)

output_did = result["did"]
print(f"Output DID: {output_did}")

In [ ]:
nevermined.assets.download(output_did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")